<a href="https://colab.research.google.com/github/aliebi/sentiment-classificatio-Naive-Bayes-Logistic-Regression-/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from fastai import *
from fastai.text import *
import sklearn.feature_extraction.text as sklearn_text

now we choose our dataset from fastai 

In [2]:
path = untar_data(URLs.IMDB_SAMPLE)
df = pd.read_csv(path/'texts.csv')
df

,label,text,is_valid
0,negative,Un-bleeping-believable! Meg Ryan doesn't even ...,False
1,positive,This is a extremely well-made film. The acting...,False
2,negative,Every once in a long while a movie will come a...,False
3,positive,Name just says it all. I watched this movie wi...,False
4,negative,This movie succeeds at being one of the most u...,False
5,negative,"From the start, you know how this movie will e...",False
6,negative,There were a lot of truly great horror movies ...,False
7,negative,I was fortunate enough to meet George Pal (and...,False
8,negative,This film is the freshman effort of Stephanie ...,False
9,negative,Greg Davis and Bryan Daly take some crazed sta...,False


here we can see data set looklike

In [0]:

movie_reviews = (TextList.from_csv(path, 'texts.csv', cols='text')
                         .split_from_df(col=2)
                         .label_from_df(cols=0))

In [4]:
movie_reviews

LabelLists;

Train: LabelList (800 items)
x: TextList
xxbos xxmaj un - xxunk - believable ! xxmaj meg xxmaj ryan does n't even look her usual xxunk lovable self in this , which normally makes me forgive her shallow xxunk acting xxunk . xxmaj hard to believe she was the producer on this dog . xxmaj plus xxmaj kevin xxmaj kline : what kind of suicide trip has his career been on ? xxmaj xxunk ... xxmaj xxunk ! ! ! xxmaj finally this was directed by the guy who did xxmaj big xxmaj xxunk ? xxmaj must be a replay of xxmaj jonestown - hollywood style . xxmaj xxunk !,xxbos xxmaj this is a extremely well - made film . xxmaj the acting , script and camera - work are all first - rate . xxmaj the music is good , too , though it is mostly early in the film , when things are still relatively xxunk . xxmaj there are no really xxunk in the cast , though several faces will be familiar . xxmaj the entire cast does an excellent job with the script . 
 
  xxmaj but it is hard to watch , because there is n

some words above are UNK like ''xxmaj'' you can see  desc of them here in https://docs.fast.ai/text.transform.html

for one pair of comment we have:


In [5]:
movie_reviews.valid.x[0], movie_reviews.valid.y[0]

(Text xxbos xxmaj this very funny xxmaj british comedy shows what might happen if a section of xxmaj london , in this case xxmaj xxunk , were to xxunk itself independent from the rest of the xxup uk and its laws , xxunk & post - war xxunk . xxmaj merry xxunk is what would happen . 
  
   xxmaj the explosion of a wartime bomb leads to the xxunk of ancient xxunk which show that xxmaj xxunk was xxunk to the xxmaj xxunk of xxmaj xxunk xxunk ago , a small historical xxunk long since forgotten . xxmaj to the new xxmaj xxunk , however , this is an unexpected opportunity to live as they please , free from any xxunk from xxmaj xxunk . 
  
   xxmaj stanley xxmaj xxunk is excellent as the minor city xxunk who suddenly finds himself leading one of the world 's xxunk xxunk . xxmaj xxunk xxmaj margaret xxmaj xxunk is a delight as the history professor who sides with xxmaj xxunk . xxmaj others in the stand - out cast include xxmaj xxunk xxmaj xxunk , xxmaj paul xxmaj xxunk , xxmaj xxunk xxmaj xxunk ,

size of train and valid data

In [6]:
len(movie_reviews.train.x), len(movie_reviews.valid.x)

(800, 200)

In [7]:
len(movie_reviews.vocab.itos), len(movie_reviews.vocab.stoi)

(6008, 19161)

stoi is all words by UNK words and itos is just words 
for examlpe:

In [0]:
unk = []
for word, num in movie_reviews.vocab.stoi.items():
    if num==0:
        unk.append(word)

In [9]:
print("num of UNK",len(unk))
len(movie_reviews.vocab.stoi) - len(movie_reviews.vocab.itos)

num of UNK 13154


13153

as you see above there are equla 

In [10]:
movie_reviews.vocab.itos[500:520]

['killed',
 'wanted',
 'entertaining',
 'based',
 'absolutely',
 'feeling',
 'truly',
 'etc',
 'heard',
 'serious',
 'run',
 'wonderful',
 'lives',
 'gives',
 'moment',
 'game',
 'documentary',
 'self',
 'several',
 'waste']

In [11]:
movie_reviews.vocab.stoi

defaultdict(int,
            {'xxunk': 0,
             'xxpad': 1,
             'xxbos': 2,
             'xxeos': 3,
             'xxfld': 4,
             'xxmaj': 5,
             'xxup': 6,
             'xxrep': 7,
             'xxwrep': 8,
             'the': 9,
             '.': 10,
             ',': 11,
             'and': 12,
             'a': 13,
             'of': 14,
             'to': 15,
             'is': 16,
             'it': 17,
             'in': 18,
             'i': 19,
             'that': 20,
             'this': 21,
             '"': 22,
             "'s": 23,
             '\n \n ': 24,
             '-': 25,
             'was': 26,
             'as': 27,
             'for': 28,
             'movie': 29,
             'with': 30,
             'but': 31,
             'film': 32,
             'you': 33,
             ')': 34,
             'on': 35,
             '(': 36,
             "n't": 37,
             'are': 38,
             'he': 39,
             'his': 40,
       

In [12]:
movie_reviews.vocab.itos[movie_reviews.vocab.stoi['xxxxxxxxxxxxxxxxxxxxxxxxxxxxx']]

'xxunk'

In [13]:
t = movie_reviews.train[0][0]
len(t.data)
t

Text xxbos xxmaj un - xxunk - believable ! xxmaj meg xxmaj ryan does n't even look her usual xxunk lovable self in this , which normally makes me forgive her shallow xxunk acting xxunk . xxmaj hard to believe she was the producer on this dog . xxmaj plus xxmaj kevin xxmaj kline : what kind of suicide trip has his career been on ? xxmaj xxunk ... xxmaj xxunk ! ! ! xxmaj finally this was directed by the guy who did xxmaj big xxmaj xxunk ? xxmaj must be a replay of xxmaj jonestown - hollywood style . xxmaj xxunk !

In [14]:
m = movie_reviews.train.x[0]
m

Text xxbos xxmaj un - xxunk - believable ! xxmaj meg xxmaj ryan does n't even look her usual xxunk lovable self in this , which normally makes me forgive her shallow xxunk acting xxunk . xxmaj hard to believe she was the producer on this dog . xxmaj plus xxmaj kevin xxmaj kline : what kind of suicide trip has his career been on ? xxmaj xxunk ... xxmaj xxunk ! ! ! xxmaj finally this was directed by the guy who did xxmaj big xxmaj xxunk ? xxmaj must be a replay of xxmaj jonestown - hollywood style . xxmaj xxunk !

now we can make count matrix and compress it in csr matrix  format

In [0]:
def get_term_doc_matrix(label_list, vocab_len):
    j_indices = []
    indptr = []
    values = []
    indptr.append(0)

    for i, doc in enumerate(label_list):
        feature_counter = Counter(doc.data)
        j_indices.extend(feature_counter.keys())
        values.extend(feature_counter.values())
        indptr.append(len(j_indices))
        
#     return (values, j_indices, indptr)

    return scipy.sparse.csr_matrix((values, j_indices, indptr),
                                   shape=(len(indptr) - 1, vocab_len),
                                   dtype=int)

In [16]:
%%time
val_term_doc = get_term_doc_matrix(movie_reviews.valid.x, len(movie_reviews.vocab.itos))

CPU times: user 45.4 ms, sys: 3.05 ms, total: 48.5 ms
Wall time: 53.1 ms


In [17]:
%%time
trn_term_doc = get_term_doc_matrix(movie_reviews.train.x, len(movie_reviews.vocab.itos))

CPU times: user 189 ms, sys: 9.28 ms, total: 199 ms
Wall time: 187 ms


we have 200 row for validation data and have 6008 features

In [18]:
val_term_doc.shape

(200, 6008)

# *we  transform data and now start naive bayes:

In [46]:

movie_reviews.y.classes
x = trn_term_doc
y = movie_reviews.train.y
val_y = movie_reviews.valid.y
positive = y.c2i['positive']
negative = y.c2i['negative']


x

<800x6008 sparse matrix of type '<class 'numpy.int64'>'
	with 112405 stored elements in Compressed Sparse Row format>

now can calculate number of positive and negative comment

In [43]:
print(np.squeeze(np.asarray(x[y.items==negative].sum(0))).shape)
print(np.squeeze(np.asarray(x[y.items==positive].sum(0))).shape)
pos=np.squeeze(np.asarray(x[y.items==positive].sum(0)))
neg=np.squeeze(np.asarray(x[y.items==negative].sum(0)))
print(pos)
print(neg)

(6008,)
(6008,)
[6471    0  383    0 ...    3    0    0    0]
[7154    0  417    0 ...    0    3    3    3]


now we have vector of pos and neagtive comment and can

In [48]:
v = movie_reviews.vocab
v.stoi['loved']

535

In [49]:
pos[535]

29

In [50]:
neg[535]

12

In [65]:
a=np.argwhere((x[:,535] > 0))[:,0]
a

array([  1,  15,  29,  69,  75,  79, 174, 185, 200, 205, 262, 296, 303, 333, 350, 351, 398, 407, 440, 489, 496, 528,
       538, 600, 602, 605, 627, 642, 657, 660, 700, 712, 729, 735, 755, 767, 785], dtype=int32)

In [66]:
b = np.argwhere(y.items==positive)[:,0]; b


array([  1,   3,  10,  11, ..., 787, 789, 790, 797])

these are positive comment that have 'loved'

In [68]:
set(a).intersection(set(b))

{1,
 29,
 69,
 75,
 79,
 174,
 185,
 262,
 296,
 333,
 350,
 407,
 440,
 489,
 496,
 528,
 538,
 602,
 627,
 657,
 660,
 712,
 735,
 755,
 785}

and can see negative comments have 'loved'

In [69]:
set(a).difference(b)

{15, 200, 205, 303, 351, 398, 600, 605, 642, 700, 729, 767}

In [70]:
movie_reviews.train.x[1].text

'xxbos xxmaj this is a extremely well - made film . xxmaj the acting , script and camera - work are all first - rate . xxmaj the music is good , too , though it is mostly early in the film , when things are still relatively xxunk . xxmaj there are no really xxunk in the cast , though several faces will be familiar . xxmaj the entire cast does an excellent job with the script . \n \n  xxmaj but it is hard to watch , because there is no good end to a situation like the one presented . xxmaj it is now xxunk to blame the xxmaj british for setting xxmaj hindus and xxmaj muslims against each other , and then xxunk xxunk them into two countries . xxmaj there is some merit in this view , but it \'s also true that no one forced xxmaj hindus and xxmaj muslims in the region to xxunk each other as they did around the time of partition . xxmaj it seems more likely that the xxmaj british simply saw the xxunk between the xxunk and were clever enough to exploit them to their own ends . \n \n  xxmaj th

In [71]:
movie_reviews.train.x[15].text

"xxbos xxmaj as if the storyline was n't depressing enough , this movie shows xxunk being xxunk graphically in a slaughterhouse for all of five minutes while the protagonist is xxunk her early life as a butcher . xxmaj weird stuff . xxmaj then there 's the core premise of the hero / heroine who goes and cuts his dick off because a he 's xxunk - ten with at work says he would have gone with him if he was a girl . xxmaj is this person a psycho , a xxunk , just a doomed queen who takes things too far ? xxmaj and what sort of xxunk childhood did he have ? xxmaj just that he did n't get adopted and had to live it out with xxunk who at first loved him and then later hated him because he was xxunk . xxmaj he tries to explain to us the reasons he did what he did , but it 's really really so hard to xxunk . xxmaj such sad and unusual self destruction . xxmaj was it supposed to be funny ? xxmaj what was it all about really ?"

# start calculate naive bayes

we should normalize 

In [72]:

p1 = np.squeeze(np.asarray(x[y.items==positive].sum(0)))
p0 = np.squeeze(np.asarray(x[y.items==negative].sum(0)))

pr1 = (p1+1) / ((y.items==positive).sum() + 1)
pr0 = (p0+1) / ((y.items==negative).sum() + 1)

r = np.log(pr1/pr0); r

array([-0.015487,  0.084839,  0.      ,  0.084839, ...,  1.471133, -1.301455, -1.301455, -1.301455])

In [0]:

biggest = np.argpartition(r, -10)[-10:]
smallest = np.argpartition(r, 10)[:10]

In [77]:
[v.itos[k] for k in biggest]

['sport',
 'davies',
 'gilliam',
 'fanfan',
 'biko',
 'felix',
 'noir',
 'jabba',
 'astaire',
 'jimmy']

In [76]:
[v.itos[k] for k in smallest]

['worst',
 'crap',
 'crater',
 'porn',
 'disappointment',
 'dog',
 'vargas',
 'naschy',
 'fuqua',
 'soderbergh']

In [78]:
(y.items==positive).mean(), (y.items==negative).mean()

(0.47875, 0.52125)

we have this formula 
![alt text](https://miro.medium.com/max/700/1*Si5V22XBbQX0fUEajrAx3Q.png)

we caluclate this two part as b , r and and then test it on validation set

In [109]:
b = np.log((y.items==positive).mean() / (y.items==negative).mean())

preds = (val_term_doc @ r + b) > 0

(preds == val_y.items).mean()


0.645

our accuracy is 0.645 for predict

or for exampe for document 2  we predict its negative

In [110]:
val = movie_reviews.valid
val[1]
preds = (val_term_doc[2] @ r + b)  

preds



array([-11.512458])

# alternative way is binarized Naive bayes

in binary we just see if word are or not , not count of them

In [0]:
# data = trn_term_doc
data = trn_term_doc.sign()
label= movie_reviews.train.y

In [116]:
data.todense()[:10,:10]

matrix([[1, 0, 1, 0, ..., 0, 0, 0, 1],
        [1, 0, 1, 0, ..., 1, 0, 0, 1],
        [1, 0, 1, 0, ..., 1, 0, 0, 1],
        [1, 0, 1, 0, ..., 0, 0, 0, 1],
        ...,
        [1, 0, 1, 0, ..., 0, 0, 0, 1],
        [1, 0, 1, 0, ..., 1, 0, 0, 1],
        [1, 0, 1, 0, ..., 0, 0, 0, 1],
        [1, 0, 1, 0, ..., 1, 0, 0, 1]])

In [0]:
negative = label.c2i['negative']
positive = label.c2i['positive']

In [0]:

p1 = np.squeeze(np.asarray(data[label.items==positive].sum(0)))
p0 = np.squeeze(np.asarray(data[label.items==negative].sum(0)))

In [0]:
pr1 = (p1+1) / ((label.items==positive).sum() + 1)
pr0 = (p0+1) / ((label.items==negative).sum() + 1)


as you see below binary has better acc 

In [128]:
r = np.log(pr1/pr0)
b = np.log((label.items==positive).mean() / (label.items==negative).mean())

preds = (val_term_doc.sign() @ r + b) > 0
(preds==val_y.items).mean()

0.695

# **logisic regression **

now we can do sentiment classification by logistic regression